# Market price monitor 
## 0. Imports

In [40]:
import urllib.request
import json, os
import pandas as pd
pd.set_option('display.max_columns', 0)
import pprint
pp = pprint.PrettyPrinter(indent=4)

FILENAME = 'push_data.csv'

BASEURL = 'http://localhost:18080/kabusapi/'
BASEURL_TEST = 'http://localhost:18081/kabusapi/'

## 1. Get token

In [1]:
from getpass import getpass
password = getpass()

 ········


In [4]:
json_data = json.dumps({'APIPassword': password}).encode('utf8')
req = urllib.request.Request(
    BASEURL+'token', 
    json_data, 
    method='POST'
)
req.add_header('Content-Type', 'application/json')

res = urllib.request.urlopen(req)
print(res.status, res.reason)

content = json.loads(res.read())
token = content['Token']

200 OK


## 2. Register brand

In [7]:
obj = { 'Symbols':
        [
            {'Symbol': '4689', 'Exchange': 1},
        ] }
json_data = json.dumps(obj).encode('utf8')

url = 'http://localhost:18080/kabusapi/register'
req = urllib.request.Request(url, json_data, method='PUT')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', token)


res = urllib.request.urlopen(req)
print(res.status, res.reason)
#for header in res.getheaders():
#    print(header)

content = json.loads(res.read())
pp.pprint(content)

200 OK
{'RegistList': [{'Exchange': 1, 'Symbol': '4689'}]}


## 3. Get PUSH

about PUSH: https://kabucom.github.io/kabusapi/ptal/push.html

In [28]:
def convertToPlain(content):
    content_plain = content.copy()
    for board_type in ['Buy', 'Sell']:
        for i in range(1, 11):
            key = board_type + str(i)
            board = content_plain.pop(key, None)
            if i == 1:
                for s in ['Sign', 'Time']:
                    content_plain[key+'_'+s] = board[s]
            for s in ['Price', 'Qty']:
                content_plain[key+'_'+s] = board[s]
    return content_plain

In [ ]:
import json, sys
import websocket
import _thread

from datetime import datetime

#data = pd.DataFrame()

def on_message(ws, message):
    print(datetime.now(), ', --- RECV MSG. ---', end='\r')
    #print(message)
    # String --> dict
    content = json.loads(message)

    # Convert to plain
    content_plain = convertToPlain(content)
    
    #pp.pprint(content_plain)
            
    # dict --> dataframe
    df = pd.DataFrame([content_plain])
    
    if not os.path.isfile(FILENAME):
        df.to_csv(FILENAME)
    else:
        df.to_csv(FILENAME, mode='a', header=False)
    
    #pd.concat([data, df], axis=1)

def on_error(ws, error):
    print('--- ERROR --- ')
    print(error)

def on_close(ws):
    print('--- DISCONNECTED --- ')

def on_open(ws):
    print('--- CONNECTED --- ')
    def run(*args):
        while(True):
            line = sys.stdin.readline()
            if line != '':
                print('closing...')
                ws.close()
    #_thread.start_new_thread(run, ())

url = 'ws://localhost:18080/kabusapi/websocket'
# websocket.enableTrace(True)
ws = websocket.WebSocketApp(
    url,
    on_message = on_message,
    on_error = on_error,
    on_close = on_close
)
ws.on_open = on_open
ws.run_forever()

--- CONNECTED --- 
